### Домашнее задание

1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. при обучении моделей обязательно использовать кроссвалидацию
3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
4. сделать выводы о том, какая модель справилась с задачей лучше других
5. (опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого). 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion, make_pipeline, Pipeline
from sklearn.metrics import confusion_matrix, precision_recall_curve

In [2]:
df = pd.read_csv('train_case2.csv', ';')
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


Разделим наши данные на тренировочную и тестовую выборки

In [3]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), 
                                                    df['cardio'], random_state=0)

К полям:
- gender, cholesterol применим OHE-кодирование
- age, height, weight, ap_hi, ap_lo - standardScaler
- gluc, smoke, alco, active - оставим пока как есть

In [4]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


from sklearn.preprocessing import StandardScaler


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

Теперь объединим все наши трансформеры с помощью FeatureUnion

In [5]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

Добавим классификатор и запустим кросс-валидацию

In [6]:
def roc_auc_show(y_true, y_pred):
    sns.set(font_scale=1.5)
    sns.set_color_codes("muted")

    plt.figure(figsize=(10, 8))
    fpr, tpr, thresholds_ = roc_curve(y_true, y_pred, pos_label=1)
    lw = 2
    plt.plot(fpr, tpr, lw=lw, label='ROC curve ')
    plt.plot([0, 1], [0, 1])
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve')
    plt.show()

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score, log_loss

In [8]:
%%time

models = [LogisticRegression(random_state=42), 
          RandomForestClassifier(random_state=42),
          AdaBoostClassifier(random_state=42),
          CatBoostClassifier(random_state=42, silent=True)]

def evaluate_model(model):
    classifier = Pipeline([
        ('features',feats),
        ('classifier', model),
    ])

    cv_scores = cross_val_score(classifier, X_train, y_train, cv=5, scoring='f1')
    cv_score = np.mean(cv_scores)

    classifier.fit(X_train, y_train)
    y_score = classifier.predict_proba(X_test)[:, 1]

    b=1
    precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
    fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
    ix = np.argmax(fscore)

    roc_auc = roc_auc_score(y_true=y_test, y_score=classifier.predict_proba(X_test)[:,1])
    
    cnf_matrix = confusion_matrix(y_test, y_score>thresholds[ix])
    
    TN = cnf_matrix[0][0]
    FN = cnf_matrix[1][0]
    TP = cnf_matrix[1][1]
    FP = cnf_matrix[0][1]
    
    TPR = TP/(TP+FN)
    FPR = FP/(FP+TN)
    TNR = TN/(FP+TN)
    TPR, FPR, TNR
    
    return fscore[ix], precision[ix], recall[ix], roc_auc, TN, FN, TP, FP, TPR, FPR, TNR

evaluation_scores = pd.DataFrame(columns=['model', 
                                          'fscore', 
                                          'precision', 
                                          'recall', 
                                          'roc_auc', 
                                          'TN',
                                          'FN',
                                          'TP',
                                          'FP',
                                          'TPR',
                                          'FPR',
                                          'TNR'])

for model in models:
    fscore, precision, recall, roc_auc, TN, FN, TP, FP, TPR, FPR, TNR = evaluate_model(model)
    to_append = {'model': str(model).split("(")[0], 
                 'fscore': fscore, 
                 'precision': precision, 
                 'recall': recall, 
                 'roc_auc': roc_auc, 'TN': TN, 'FN': FN, 'TP': TP, 'FP': FP, 'TPR': TPR, 'FPR': FPR, 'TNR': TNR}
    evaluation_scores = evaluation_scores.append(to_append, ignore_index=True)
    
evaluation_scores

CPU times: user 5min 4s, sys: 5.71 s, total: 5min 9s
Wall time: 1min 56s


,model,fscore,precision,recall,roc_auc,TN,FN,TP,FP,TPR,FPR,TNR
0,LogisticRegression,0.730323,0.647431,0.837558,0.784035,4861,1411,7269,3959,0.837442,0.448866,0.551134
1,RandomForestClassifier,0.718863,0.642669,0.815553,0.771037,4991,1663,7017,3829,0.808410,0.434127,0.565873
2,AdaBoostClassifier,0.737766,0.692471,0.789401,0.794572,5777,1829,6851,3043,0.789286,0.345011,0.654989
3,<catboost.core.CatBoostClassifier object at 0x...,0.739621,0.689686,0.797350,0.801092,5706,1760,6920,3114,0.797235,0.353061,0.646939


Вывод: Если судить строго по f-score, не вдаваясь в смысл задачи, то, конечно, catboost побеждает как лучшая модель с данным показателем. Однако, в задачах, где цена ошибки за предсказание False Negative (второго рода) велика, то я бы оринетировался на наивысший показатель recall. В этой задаче среди всех моделей наилучший показатель FN, и recall - у LogisticRegression. Иными словами, FN - значит ложноотрицательно заключение, модель посчитала человека здоровым, а у него было сердечно-сосудистое заболевание. У LogisticRegression этот показатель наименьший. Все, конечно же, зависит от поставленной задачи, имеющихся средств и бюджета, и желаемого результата.

Опциональный вопрос (задание 5): Конечно же для несбалансированных датасетов лучше подходит precision_recall_curve, precision и recall специально предназначены для обнаружения редких событий и весьма полезны в таких сценариях, где датасет имбалансен. precision и recall покажут, что наш классификатор плохо справляется, если он неправильно классифицирует большую часть или весь класс меньшинства.